# Goals

- uses .shp ecoregion files and .shp from .gpx file
- makes intersection 
- count proportions of ecoregions that appear in intersection
- store in dictionary
- print results

# Still need to figure out
- eventually, make this into a function that takes in just the .gpx file 

In [7]:
import sys
sys.executable

'C:\\Users\\A02315404\\Anaconda3\\envs\\raster_proj\\python.exe'

In [2]:
import arcpy
import os
import classtools
arcpy.CheckOutExtension('spatial')

data_folder = r'C:\Users\A02315404\Documents\python\frolic_analysis\data'
conv_data_folder = r'C:\Users\A02315404\Documents\python\frolic_analysis\data_converted'

filename = "Temple_Fork"

#shape/image data
eco3 = os.path.join(data_orig_folder, "ecoregionsL3.shp")
eco4 = os.path.join(data_folder, "ecoregionsL4.shp")
econew=  os.path.join(data_folder, "ut_eco_l4.shp")
arcpy.env.workspace = data_folder
int_folder = r'C:\Users\A02315404\Documents\python\frolic_analysis\intersections'

In [16]:
user_input="C:/Users/A02315404/Downloads/Fischer_towers.gpx"

#modules (environ = raster_proj)
import arcpy
import os
#import classtools
arcpy.CheckOutExtension('spatial')
from arcpy import env
from arcpy.sa import *
import rasterio
import csv
import numpy as np
import gdal

arcpy.overwriteoutput = True

filename = os.path.split(user_input)[1]
filename = filename.replace(".gpx", "")

template = r'C:\Users\A02315404\Documents\python\frolic_analysis\data\ecoregionsL4.shp'

#folders
conv_data_folder = r'C:\Users\A02315404\Documents\python\frolic_analysis\data_converted'
data_folder = r'C:\Users\A02315404\Documents\python\frolic_analysis\data'
data_orig_folder = r'C:\Users\A02315404\Documents\python\frolic_analysis\data_orig'
int_folder = r'C:\Users\A02315404\Documents\python\frolic_analysis\intersections'
in_folder = r'C:\Users\A02315404\Documents\python\frolic_analysis\data_gpx_files'
out_folder = r'C:\Users\A02315404\Documents\python\frolic_analysis\data_converted'

arcpy.env.workspace = data_folder

#shape/image data
rock = os.path.join(data_orig_folder, "rock.tif")
nlcd = os.path.join(data_orig_folder, "nlcd.tif")
nlcd_key = os.path.join(data_orig_folder, "nlcd_key.csv")
rock_key = os.path.join(data_orig_folder, "rock_key.csv")
eco3 = os.path.join(data_orig_folder, "ecoregionsL3.shp")
eco4 = os.path.join(data_orig_folder, "ecoregionsL4.shp")


#trail (from user input)
trail =  os.path.join(conv_data_folder, filename) + ".shp"
output_name_shp = os.path.join(out_folder, filename)+ ".shp"
output_name_tif = os.path.join(out_folder, filename)+ ".tif"
out_name_nlcd=os.path.join(int_folder, filename) + "_nlcd.tif"
out_name_rock=os.path.join(int_folder, filename) + "_rock.tif"



In [10]:
trail =  os.path.join(conv_data_folder, filename) + ".shp"
print(trail)

C:\Users\A02315404\Documents\python\frolic_analysis\data_converted\Fischer_towers.shp


In [15]:
data_list = [trail,eco3,eco4]
print(data_list)
out_name=os.path.join(int_folder, filename) + ".shp"

['C:\\Users\\A02315404\\Documents\\python\\frolic_analysis\\data_converted\\Fischer_towers.shp', 'C:\\Users\\A02315404\\Documents\\python\\frolic_analysis\\data\\ecoregionsL3.shp', 'C:\\Users\\A02315404\\Documents\\python\\frolic_analysis\\data\\ecoregionsL4.shp']


In [12]:
arcpy.analysis.Intersect(in_features=data_list, 
                         out_feature_class=out_name, 
                         )

ExecuteError: Failed to execute. Parameters are not valid.
ERROR 000732: Input Features: Dataset Fischer_towers does not exist or is not supported
Failed to execute (Intersect).


In [16]:
fields = (arcpy.ListFields (out_name))
field_names = []

for field in fields:
        field_names.append(field.name)
        
print(field_names)

['FID', 'Shape', 'FID_Temple', 'Id', 'Name', 'Descript', 'Type', 'Comment', 'Symbol', 'DateTimeS', 'Elevation', 'FID_ecoreg', 'l3_key', 'na_l1name', 'na_l1code', 'na_l3name', 'shape_leng', 'l1_key', 'na_l3code', 'shape_area', 'us_l3code', 'na_l2name', 'us_l3name', 'na_l2code', 'l2_key', 'FID_ecor_1', 'l3_key_1', 'us_l4code', 'na_l1nam_1', 'na_l1cod_1', 'na_l3nam_1', 'shape_le_1', 'us_l4name', 'l1_key_1', 'na_l3cod_1', 'shape_ar_1', 'us_l3cod_1', 'na_l2nam_1', 'us_l3nam_1', 'na_l2cod_1', 'l4_key', 'l2_key_1']


-module 5, value tables
-this has >10000 pnts....

# get unique values in each column

In [57]:
def unique_values(table , field):
    with arcpy.da.SearchCursor(table, [field]) as cursor:
        return sorted({row[0] for row in cursor})

eco4_vals = unique_values(out_name , 'us_l4name')
eco3_vals = unique_values(out_name , 'na_l3name')
eco2_vals =unique_values(out_name , 'na_l2name')
eco1_vals =unique_values(out_name , 'na_l1name')
print(eco4_vals,eco3_vals,eco2_vals,eco1_vals)

['Semiarid Foothills', 'Wasatch Montane Zone'] ['Wasatch and Uinta Mountains'] ['WESTERN CORDILLERA'] ['NORTHWESTERN FORESTED MOUNTAINS']


# make dictionary of results using loops

In [61]:
eco4_dict = {}
eco3_dict = {}
eco2_dict = {}
eco1_dict = {}

count_all = int(arcpy.management.GetCount(out_name).getOutput(0))
count=0
for eco4_name in eco4_vals:
    subset_eco4 = arcpy.management.SelectLayerByAttribute(
        in_layer_or_view= out_name, 
        where_clause=f"us_l4name LIKE  '{eco4_vals[count]}' "
    )
    percent_eco4 = ((int(arcpy.management.GetCount(subset_eco4).getOutput(0)))/int(count_all)*100 )
    eco4_dict[eco4_vals[count]] = percent_eco4
    count=count+1
count=0
for eco3_name in eco3_vals:
    subset_eco3 = arcpy.management.SelectLayerByAttribute(
        in_layer_or_view= out_name, 
        where_clause=f"na_l3name LIKE  '{eco3_vals[count]}' "
    )
    percent_eco3 = ((int(arcpy.management.GetCount(subset_eco3).getOutput(0)))/int(count_all)*100 )
    eco3_dict[eco3_vals[count]] = percent_eco3
    count=count+1
count=0
for eco2_name in eco2_vals:
    subset_eco2 = arcpy.management.SelectLayerByAttribute(
        in_layer_or_view= out_name, 
        where_clause=f"na_l2name LIKE  '{eco2_vals[count]}' "
    )
    percent_eco2 = ((int(arcpy.management.GetCount(subset_eco2).getOutput(0)))/int(count_all)*100 )
    eco2_dict[eco2_vals[count]] = percent_eco2
    count=count+1
    
count=0
for eco1_name in eco1_vals:
    subset_eco1 = arcpy.management.SelectLayerByAttribute(
        in_layer_or_view= out_name, 
        where_clause=f"na_l1name LIKE  '{eco1_vals[count]}' "
    )
    percent_eco1 = ((int(arcpy.management.GetCount(subset_eco1).getOutput(0)))/int(count_all)*100 )
    eco1_dict[eco1_vals[count]] = percent_eco1
    count=count+1

{'Semiarid Foothills': 95.25910839556461, 'Wasatch Montane Zone': 4.740891604435393}
{'Wasatch and Uinta Mountains': 100.0, 'magic mountains': 0.0}
{'WESTERN CORDILLERA': 100.0}
{'NORTHWESTERN FORESTED MOUNTAINS': 100.0}


# print out a summary statement

In [70]:
#print( f"Your adventure took you through the '{eco3_dict[0]} ")
count=0
for eco in eco2_dict.items():
    if count==0:
        print(f"Your adventure took you through the {eco[0]} ( {round(eco[1])} % ) ") 
    if count!=0:
        print(f"and the {eco[0]} ( {round(eco[1])} % ) ") 
    count=count+1

#for eco in eco2_dict.items():
#    print(f"as well as through the {eco[0]} ( {eco[1]} % ) ") 
#for eco in eco3_dict.items():
#    print(f"Your adventure took you through the {eco[0]} ( {eco[1]} % ) ") 
count=0
for eco in eco4_dict.items():
    if count==0:
        print(f"specifically through the {(eco[0])} ( {round(eco[1])} % ) ")
    if count!=0:
        print(f" and the {(eco[0])} ( {round(eco[1])} % ) ") 
    count=count+1


Your adventure took you through the NORTHWESTERN FORESTED MOUNTAINS ( 100 % ) 
specifically through the Semiarid Foothills ( 95 % ) 
 and the Wasatch Montane Zone ( 5 % ) 
